# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Frameworks</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


## Construindo um Classificador de Sentimentos com PyTorch

Tudo o que expressamos (verbalmente ou por escrito) carrega enormes quantidades de informação. O tópico que escolhemos, nosso tom, nossa seleção de palavras, tudo acrescenta algum tipo de informação que pode ser interpretada e com o valor extraído dela. Em teoria, podemos entender e até prever o comportamento humano usando essas informações.

Mas há um problema: uma pessoa pode gerar centenas ou milhares de palavras em uma declaração, cada sentença com sua complexidade correspondente. Se você deseja dimensionar e analisar várias centenas, milhares ou milhões de pessoas ou declarações em uma determinada região, a situação é incontrolável.


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [1]:
# Imports
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm, tqdm_notebook

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

torch   1.4.0
pandas  1.0.3
sklearn 0.22.2
Data Science Academy


In [2]:
# Define o device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Carregando e Explorando os Dados

Usaremos um dataset disponível publicamente em https://www.imdb.com/interfaces/.

Os labels de sentimentos foram extraídos do portal: https://ai.stanford.edu/~amaas/data/sentiment/

In [3]:
# Carrega o dataset
nomes_colunas = ['Review', 'Sentimento']
dados_filmes = pd.read_csv('dados/imdb_reviews.csv', sep = '\t', names = nomes_colunas)

In [4]:
# Visualiza
dados_filmes.head()

,Review,Sentimento
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [5]:
# Shape
dados_filmes.shape

(748, 2)

In [6]:
# Verificando a proporção de sentimentos
dados_filmes['Sentimento'].value_counts()

1    386
0    362
Name: Sentimento, dtype: int64

## Representação Bag-of-Words

![](imagens/bag2.png)

![](imagens/bag1.jpeg)

![](imagens/bag3.png)

![](imagens/bag4.png)

## Manipulação de Texto

Começamos criando um "vetorizador".

Convertemos uma coleção de documentos de texto em uma matriz de contagens de tokens.

Essa implementação produz uma representação esparsa das contagens usando scipy.sparse.csr_matrix.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [7]:
# Criamos um vectorizer
vectorizer = CountVectorizer(stop_words = 'english', max_df = 0.99, min_df = 0.005)
vectorizer

CountVectorizer(max_df=0.99, min_df=0.005, stop_words='english')

In [8]:
# Extraímos as sequências do texto aplicando o vetorizador
sequences = vectorizer.fit_transform(dados_filmes.Review.tolist())
sequences

<748x320 sparse matrix of type '<class 'numpy.int64'>'
	with 2931 stored elements in Compressed Sparse Row format>

In [9]:
# Visualiza como dataframe
print(pd.DataFrame(sequences).head(3))

                                                   0
0    (0, 248)\t1\n  (0, 185)\t1\n  (0, 183)\t1\n ...
1    (0, 162)\t1\n  (0, 41)\t1\n  (0, 14)\t1\n  (...
2    (0, 183)\t1\n  (0, 28)\t1\n  (0, 305)\t1\n  ...


In [10]:
# Organiza os labels (sentimentos)
labels = dados_filmes.Sentimento.tolist()

In [11]:
# Visualiza uma amostra
labels[:5]

[0, 0, 0, 0, 1]

In [12]:
# Criamos o saco de palavras (vocabulário)
token2idx = vectorizer.vocabulary_

In [13]:
# Tipo
type(token2idx)

dict

In [14]:
# Total
print(len(token2idx))

320


In [15]:
# Visualiza
token2idx

{'10': 0,
 '90': 1,
 'absolutely': 2,
 'acting': 3,
 'action': 4,
 'actor': 5,
 'actors': 6,
 'actually': 7,
 'adorable': 8,
 'age': 9,
 'amazing': 10,
 'annoying': 11,
 'art': 12,
 'attempt': 13,
 'audience': 14,
 'avoid': 15,
 'away': 16,
 'awful': 17,
 'bad': 18,
 'barely': 19,
 'beautiful': 20,
 'beginning': 21,
 'believable': 22,
 'believe': 23,
 'best': 24,
 'better': 25,
 'big': 26,
 'bit': 27,
 'black': 28,
 'book': 29,
 'bored': 30,
 'boring': 31,
 'brilliant': 32,
 'budget': 33,
 'came': 34,
 'camera': 35,
 'care': 36,
 'cast': 37,
 'casting': 38,
 'certainly': 39,
 'character': 40,
 'characters': 41,
 'cheap': 42,
 'chemistry': 43,
 'child': 44,
 'cinema': 45,
 'cinematography': 46,
 'classic': 47,
 'clever': 48,
 'come': 49,
 'comedy': 50,
 'comes': 51,
 'completely': 52,
 'consider': 53,
 'convincing': 54,
 'cool': 55,
 'couldn': 56,
 'cover': 57,
 'crap': 58,
 'created': 59,
 'cult': 60,
 'dance': 61,
 'day': 62,
 'death': 63,
 'definitely': 64,
 'depth': 65,
 'dialogue':

In [16]:
# Quantas vezes a palavra "movie" aparece nas avaliações?
token2idx['movie']

183

In [17]:
# E a palavra good?
token2idx['good']

115

In [18]:
# Para facilitar nosso trabalho, vamos inverter chaves e colunas em nosso dicionário
idx2token = {idx: token for token, idx in token2idx.items()}

In [19]:
# Tipo
type(idx2token)

dict

In [20]:
# Total
print(len(idx2token))

320


In [21]:
# Visualiza
idx2token

{0: '10',
 1: '90',
 2: 'absolutely',
 3: 'acting',
 4: 'action',
 5: 'actor',
 6: 'actors',
 7: 'actually',
 8: 'adorable',
 9: 'age',
 10: 'amazing',
 11: 'annoying',
 12: 'art',
 13: 'attempt',
 14: 'audience',
 15: 'avoid',
 16: 'away',
 17: 'awful',
 18: 'bad',
 19: 'barely',
 20: 'beautiful',
 21: 'beginning',
 22: 'believable',
 23: 'believe',
 24: 'best',
 25: 'better',
 26: 'big',
 27: 'bit',
 28: 'black',
 29: 'book',
 30: 'bored',
 31: 'boring',
 32: 'brilliant',
 33: 'budget',
 34: 'came',
 35: 'camera',
 36: 'care',
 37: 'cast',
 38: 'casting',
 39: 'certainly',
 40: 'character',
 41: 'characters',
 42: 'cheap',
 43: 'chemistry',
 44: 'child',
 45: 'cinema',
 46: 'cinematography',
 47: 'classic',
 48: 'clever',
 49: 'come',
 50: 'comedy',
 51: 'comes',
 52: 'completely',
 53: 'consider',
 54: 'convincing',
 55: 'cool',
 56: 'couldn',
 57: 'cover',
 58: 'crap',
 59: 'created',
 60: 'cult',
 61: 'dance',
 62: 'day',
 63: 'death',
 64: 'definitely',
 65: 'depth',
 66: 'dialog

Vamos resumir tudo que fizemos em uma função.

In [22]:
# Definimos uma classe para extrair as frases
class Sequences():
    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words = 'english', max_df = 0.99, min_df = 0.005)
        self.sequences = self.vectorizer.fit_transform(dados_filmes.Review.tolist())
        self.labels = dados_filmes.Sentimento.tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]
    
    def __len__(self):
        return self.sequences.shape[0]

In [23]:
# Extrai as frases do dataset e cria a matriz de dados, como os tokens, contagens e labels
dados_frases = Sequences()

In [24]:
# Confere o shape
print(dados_frases[5][0].shape)

(1, 320)


In [25]:
# Prepara os dados para treinamento no formato PyTorch
train_loader = DataLoader(dados_frases, batch_size = 4096)
train_loader

## Definição e Construção do Modelo

Camada 1: $$x_1 = W_1 X + b_1$$
Função de Ativação: $$h_1 = \textrm{Relu}(x_1)$$
Camada 2: $$x_2 = W_2 h_1 + b_2$$
Saída: $$p = \sigma(x_2)$$
Loss: $$L = −(ylog(p)+(1−y)log(1−p))$$
Gradiente: 
$$\frac{\partial }{\partial W_1}L(W_1, b_1, W_2, b_2) = \frac{\partial L}{\partial p}\frac{\partial p}{\partial x_2}\frac{\partial x_2}{\partial h_1}\frac{\partial h_1}{\partial x_1}\frac{\partial x_1}{\partial W_1}$$

Atualização de Parâmetros:
$$W_1 = W_1 - \alpha \frac{\partial L}{\partial W_1}$$

In [26]:
# Classificador
class BagOfWordsClassifier(nn.Module):
    
    # Método construtor para inicializar os atributos
    def __init__(self, vocab_size, hidden1, hidden2):
        super(BagOfWordsClassifier, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
    
    # Método para a passada para a frente (forward)
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [27]:
# Cria o modelo
modelo = BagOfWordsClassifier(len(dados_frases.token2idx), 128, 64)

In [28]:
# Visualiza
modelo

BagOfWordsClassifier(
  (fc1): Linear(in_features=320, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

Para a função de perda (loss) usaremos Binary Cross Entropy.

In [29]:
# Define a função de perda
criterion = nn.BCEWithLogitsLoss()

Para o otimizador usaremos o algoritmo ADAM.

In [30]:
# Adam dinamicamente altera a taxa de aprendizagem
optimizer = optim.Adam([p for p in modelo.parameters() if p.requires_grad], lr = 0.001)

Agora treinamos o modelo.

In [32]:
# Treinamento

# Instância de treinamento do modelo
modelo.train()

# Lista para armazenar os erros a cada passada de treinamento
train_losses = []

# Número de épocas
epochs = 100

# Loop de treinamento
for epoch in range(epochs): 
    
    # Barra de progresso
    progress_bar = tqdm_notebook(train_loader, leave = False)
    
    # Listas de controle
    losses = []
    total = 0
    
    # Loop
    for inputs, target in progress_bar:
        
        # Modelo
        modelo.zero_grad()

        # Saída (previsão do modelo)
        output = modelo(inputs)
        
        # Cálculo do erro
        loss = criterion(output.squeeze(), target.float())
        
        # Instância do Backpropagation
        loss.backward()
        
        # Prepara atualização dos parâmetros (coeficientes)    
        nn.utils.clip_grad_norm_(modelo.parameters(), 3)

        # Executa o otimizador
        optimizer.step()
        
        # Atualiza a barra de progresso
        progress_bar.set_description(f'\nErro do Modelo: {loss.item():.3f}')
        
        # Erros e total
        losses.append(loss.item())
        total += 1
    
    # Erro da epoch
    epoch_loss = sum(losses) / total
    
    # Erro de treinamento
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tErro em Treinamento: {epoch_loss:.3f}')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch #1	Erro em Treinamento: 0.668


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch #2	Erro em Treinamento: 0.664


  0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch #98	Erro em Treinamento: 0.100


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch #99	Erro em Treinamento: 0.099


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch #100	Erro em Treinamento: 0.098


## Previsões de Sentimentos

In [34]:
# Função para prever o sentimento
def predict_sentiment(text):
    
    # Carrega o modelo
    modelo.eval()
    
    # Extrai as previsões do modelo
    with torch.no_grad():
        
        # Texto recebido como parâmetro convertido para vetor
        test_vector = torch.LongTensor(dados_frases.vectorizer.transform([text]).toarray())

        # Previsão
        output = modelo(test_vector)
        
        # Gera a previsão final como probabilidade
        prediction = torch.sigmoid(output).item()

        # Checa a probabilidade com limite de 0.5
        if prediction >= 0.5:
            print(f'{prediction:0.3}: Sentimento Positivo')
        else:
            print(f'{prediction:0.3}: Sentimento Negativo')

Primeiro calculamos o valor sigmóide (entre 0 e 1) para a previsão.

Se for maior ou igual que 0,5 classificamos como sentimento Positivo e se for menor que 0.5 classificamos como sentimento Negativo. 

Vamos testar o classificador de sentimentos.

In [35]:
# Texto de avaliação de filme
test_text = """
This poor excuse for a movie is terrible. It has been 'so good it's bad' for a
while, and the high ratings are a good form of sarcasm, I have to admit. But
now it has to stop. Technically inept, spoon-feeding mundane messages with the
artistic weight of an eighties' commercial, hypocritical to say the least, it
deserves to fall into oblivion. Mr. Derek, I hope you realize you are like that
weird friend that everybody know is lame, but out of kindness and Christian
duty is treated like he's cool or something. That works if you are a good
decent human being, not if you are a horrible arrogant bully like you are. Yes,
Mr. 'Daddy' Derek will end on the history books of the internet for being a
delusional sour old man who thinks to be a good example for kids, but actually
has a poster of Kim Jong-Un in his closet. Destroy this movie if you all have a
conscience, as I hope IHE and all other youtube channel force-closed by Derek
out of SPITE would destroy him in the courts.This poor excuse for a movie is
terrible. It has been 'so good it's bad' for a while, and the high ratings are
a good form of sarcasm, I have to admit. But now it has to stop. Technically
inept, spoon-feeding mundane messages with the artistic weight of an eighties'
commercial, hypocritical to say the least, it deserves to fall into oblivion.
Mr. Derek, I hope you realize you are like that weird friend that everybody
know is lame, but out of kindness and Christian duty is treated like he's cool
or something. That works if you are a good decent human being, not if you are a
horrible arrogant bully like you are. Yes, Mr. 'Daddy' Derek will end on the
history books of the internet for being a delusional sour old man who thinks to
be a good example for kids, but actually has a poster of Kim Jong-Un in his
closet. Destroy this movie if you all have a conscience, as I hope IHE and all
other youtube channel force-closed by Derek out of SPITE would destroy him in
the courts.
"""

# Previsão
predict_sentiment(test_text)

1.0: Sentimento Positivo


In [36]:
# Texto de avaliação de filme
test_text = """
Cool Cat Saves The Kids is a symbolic masterpiece directed by Derek Savage that
is not only satirical in the way it makes fun of the media and politics, but in
the way in questions as how we humans live life and how society tells us to
live life.

Before I get into those details, I wanna talk about the special effects in this
film. They are ASTONISHING, and it shocks me that Cool Cat Saves The Kids got
snubbed by the Oscars for Best Special Effects. This film makes 2001 look like
garbage, and the directing in this film makes Stanley Kubrick look like the
worst director ever. You know what other film did that? Birdemic: Shock and
Terror. Both of these films are masterpieces, but if I had to choose my
favorite out of the 2, I would have to go with Cool Cat Saves The Kids. It is
now my 10th favorite film of all time.

Now, lets get into the symbolism: So you might be asking yourself, Why is Cool
Cat Orange? Well, I can easily explain. Orange is a color. Orange is also a
fruit, and its a very good fruit. You know what else is good? Good behavior.
What behavior does Cool Cat have? He has good behavior. This cannot be a
coincidence, since cool cat has good behavior in the film.

Now, why is Butch The Bully fat? Well, fat means your wide. You wanna know who
was wide? Hitler. Nuff said this cannot be a coincidence.

Why does Erik Estrada suspect Butch The Bully to be a bully? Well look at it
this way. What color of a shirt was Butchy wearing when he walks into the area?
I don't know, its looks like dark purple/dark blue. Why rhymes with dark? Mark.
Mark is that guy from the Room. The Room is the best movie of all time. What is
the opposite of best? Worst. This is how Erik knew Butch was a bully.

and finally, how come Vivica A. Fox isn't having a successful career after
making Kill Bill.

I actually can't answer that question.

Well thanks for reading my review.
"""

# Previsão
predict_sentiment(test_text)

1.0: Sentimento Positivo


In [37]:
# Texto de avaliação de filme
test_text = """
What the heck is this ? There is not one redeeming quality about this terrible
and very poorly done "movie". I can't even say that it's a "so bad it's good
movie".It is undeniably pointless to address all the things wrong here but
unfortunately even the "life lessons" about bullies and stuff like this are so
wrong and terrible that no kid should hear them.The costume is also horrible
and the acting...just unbelievable.No effort whatsoever was put into this thing
and it clearly shows,I have no idea what were they thinking or who was it even
meant for. I feel violated after watching this trash and I deeply recommend you
stay as far away as possible.This is certainly one of the worst pieces of c***
I have ever seen.
"""

# Previsão
predict_sentiment(test_text)

2.78e-14: Sentimento Negativo


In [38]:
# Texto de avaliação de filme
test_text = """
Don't let any bullies out there try and shape your judgment on this gem of a
title.

Some people really don't have anything better to do, except trash a great movie
with annoying 1-star votes and spread lies on the Internet about how "dumb"
Cool Cat is.

I wouldn't be surprised to learn if much of the unwarranted negativity hurled
at this movie is coming from people who haven't even watched this movie for
themselves in the first place. Those people are no worse than the Butch the
Bully, the film's repulsive antagonist.

As it just so happens, one of the main points of "Cool Cat Saves the Kids" is
in addressing the attitudes of mean naysayers who try to demean others who
strive to bring good attitudes and fun vibes into people's lives. The message
to be learned here is that if one is friendly and good to others, the world is
friendly and good to one in return, and that is cool. Conversely, if one is
miserable and leaving 1-star votes on IMDb, one is alone and doesn't have any
friends at all. Ain't that the truth?

The world has uncovered a great, new, young filmmaking talent in "Cool Cat"
creator Derek Savage, and I sure hope that this is only the first of many
amazing films and stories that the world has yet to appreciate.

If you are a cool person who likes to have lots of fun, I guarantee that this
is a movie with charm that will uplift your spirits and reaffirm your positive
attitudes towards life.
"""

# Previsão
predict_sentiment(test_text)

1.0: Sentimento Positivo


# Fim